In [11]:
import gurobipy as gp
from gurobipy import GRB

def solve_container_loading():
    try:
        L, W, H = 12, 8, 8

        boxes = [
            (6, 3, 2), (6, 3, 2),  # Boxes 1-2
            (6, 4, 3), (6, 4, 3), (6, 4, 3), (6, 4, 3), (6, 4, 3), # Boxes 3-7
            (8, 3, 2), (8, 3, 2), (8, 3, 2), # Boxes 8-10
            (4, 3, 2), (4, 3, 2), # Boxes 11-12
            (4, 4, 3), (4, 4, 3), (4, 4, 3)  # Boxes 13-15
        ]
        
        m = len(boxes)

        values = [l*w*h/(L*W*H) for l,w,h in boxes]
        
        M = L + W + H 
        
        # Position of the front-left-bottom corner of the container 
        # As per the paper's logic, this should be a large number to separate
        # the container space from the space for unpacked boxes.
        Xo, Yo, Zo = 0, 0, 0

        # Create a new model
        model = gp.Model("ContainerLoading")

        # --- 3. Decision Variables  ---

        # p_i: 1 if box i is packed, 0 otherwise
        p = model.addVars(m, vtype=GRB.BINARY, name="p")

        # (x_i, y_i, z_i): front-left-bottom corner coordinates of box i
        x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
        y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
        z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")


        # a_ij, b_ij, c_ij, d_ij, e_ij, f_ij: relative position indicators
        # These are indexed by pairs (i, j) where i < j
        pairs = [(i, j) for i in range(m) for j in range(m) if i < j]
        a = model.addVars(pairs, vtype=GRB.BINARY, name="a")
        b = model.addVars(pairs, vtype=GRB.BINARY, name="b")
        c = model.addVars(pairs, vtype=GRB.BINARY, name="c")
        d = model.addVars(pairs, vtype=GRB.BINARY, name="d")
        e = model.addVars(pairs, vtype=GRB.BINARY, name="e")
        f = model.addVars(pairs, vtype=GRB.BINARY, name="f")
        
        for i in range(m):
            li, wi, hi = boxes[i]

            # Manter as caixas dentro do contêiner
            model.addConstr(x[i] >= (Xo * p[i]))
            model.addConstr(y[i] >= (Yo * p[i]))
            model.addConstr(z[i] >= (Zo * p[i]))
            model.addConstr((x[i] + li) <= (Xo + L))
            model.addConstr((y[i] + wi) <= (Yo + W))
            model.addConstr((z[i] + hi) <= (Zo + H))

            for j in range(i + 1, m):
                lj, wj, hj = boxes[j]

                # Garantir não sobreposição
                model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
                model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
                model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
                model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
                model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
                model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

                # Relação espacial por par
                model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)
        
        
        

        # --- 4. Objective Function ---
        # Maximize the total value of packed boxes 
        model.setObjective(gp.quicksum(values[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

        
        

        # --- 6. Solve the Model ---
        model.setParam('MIPGap', 1) 
        model.optimize()
        

        packed_volume = model.ObjVal
        container_volume = L * W * H
        utilization = (packed_volume / container_volume) * 100
        print(f"Total volume of packed boxes: {packed_volume}")
        print(f"Container volume utilization: {utilization:.2f}%\n")

        packed_boxes_indices = [i for i in range(m) if p[i].X > 0.5]
        unpacked_boxes_indices = [i for i in range(m) if p[i].X < 0.5]

        print(f"Packed {len(packed_boxes_indices)} of {m} boxes.")
        print("-" * 30)
        for i in packed_boxes_indices:
            # Adjust coordinates to be relative to the container's origin
            rel_x = x[i].X - Xo
            rel_y = y[i].X - Yo
            rel_z = z[i].X - Zo
            print(f"Box {i+1}: Packed at (x={rel_x:.2f}, y={rel_y:.2f}, z={rel_z:.2f})")

        if unpacked_boxes_indices:
            print("\nUnpacked boxes:")
            print(f"Indices: {[i+1 for i in unpacked_boxes_indices]}")
            
        tipo_dict = {}
        tipo_counter = 1
        for box in boxes:
            if box not in tipo_dict:
                tipo_dict[box] = tipo_counter
                tipo_counter += 1

        # Imprimir saída no formato correto
        print(f"{L} {W} {H}")  # primeira linha: dimensões do contêiner

        for i in range(m):
            li, wi, hi = boxes[i]
            x_i = x[i].X if p[i].X > 0.5 else 0
            y_i = y[i].X if p[i].X > 0.5 else 0
            z_i = z[i].X if p[i].X > 0.5 else 0
            tipo = tipo_dict[boxes[i]]  # tipo real baseado nas dimensões
            cliente = 1
            print(f"{x_i} {y_i} {z_i} {li} {wi} {hi} {tipo} {cliente}")

        with open('dados-meu.txt','w') as f:
            f.write(f"{L} {W} {H}\n")  # primeira linha: dimensões do contêiner
            for i in range(m):
                if p[i].X > 0.5:
                    li, wi, hi = boxes[i]
                    x_i, y_i, z_i = x[i].X, y[i].X, z[i].X
                    tipo = tipo_dict[boxes[i]]  # tipo baseado nas dimensões
                    cliente = 1
                    f.write(f"{x_i} {y_i} {z_i} {li} {wi} {hi} {tipo} {cliente}\n")

    except gp.GurobiError as e:
        print(f"Error code {e.errno}: {e}")

# Run the solver
solve_container_loading()

Set parameter MIPGap to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  1

Optimize a model with 825 rows, 690 columns and 2820 nonzeros
Model fingerprint: 0x121895aa
Variable types: 45 continuous, 645 integer (645 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e-02, 9e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 0.0468750
Presolve removed 138 rows and 48 columns
Presolve time: 0.00s
Presolved: 687 rows, 642 columns, 2538 nonzeros
Variable types: 45 continuous, 597 integer (597 binary)

Root relaxation: objective 1.000000e+00, 223 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/N

ahmed

In [12]:
import gurobipy as gp
from gurobipy import GRB

# Dados do contêiner
L, W, H = 12, 8, 8
Xo, Yo, Zo = 0, 0, 0
M = 50  # Número grande o suficiente

# Dados das caixas (li, wi, hi)
boxes = [
    (6, 3, 2), (6, 3, 2),              # caixas 1-2
    (6, 4, 3), (6, 4, 3), (6, 4, 3),   # caixas 3-7
    (6, 4, 3), (6, 4, 3),
    (8, 3, 2), (8, 3, 2), (8, 3, 2),   # caixas 8-10
    (4, 3, 2), (4, 3, 2),              # caixas 11-12
    (4, 4, 3), (4, 4, 3), (4, 4, 3)    # caixas 13-15
]

m = len(boxes)
vi = [(l * w * h)/(L*W*H) for l, w, h in boxes]

model = gp.Model("container_loading")

# Variáveis
p = model.addVars(m, vtype=GRB.BINARY, name="p")
x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")

a = model.addVars(m, m, vtype=GRB.BINARY, name="a")
b = model.addVars(m, m, vtype=GRB.BINARY, name="b")
c = model.addVars(m, m, vtype=GRB.BINARY, name="c")
d = model.addVars(m, m, vtype=GRB.BINARY, name="d")
e = model.addVars(m, m, vtype=GRB.BINARY, name="e")
f = model.addVars(m, m, vtype=GRB.BINARY, name="f")

# Função objetivo
model.setObjective(gp.quicksum(vi[i] * p[i] for i in range(m)), GRB.MAXIMIZE)
# primary_objective = gp.quicksum(vi[i] * p[i] for i in range(m))
#secondary_objective = gp.quicksum(p[i] for i in range(m))
#epsilon = 1 / (m + 1)
#model.setObjective(primary_objective + epsilon * secondary_objective, GRB.MAXIMIZE)

# Restrições
for i in range(m):
    li, wi, hi = boxes[i]

    # Manter as caixas dentro do contêiner
    model.addConstr(x[i] >= (Xo * p[i]))
    model.addConstr(y[i] >= (Yo * p[i]))
    model.addConstr(z[i] >= (Zo * p[i]))
    model.addConstr((x[i] + li) <= (Xo + L))
    model.addConstr((y[i] + wi) <= (Yo + W))
    model.addConstr((z[i] + hi) <= (Zo + H))

    for j in range(i + 1, m):
        lj, wj, hj = boxes[j]

        # Garantir não sobreposição
        model.addConstr((x[i] + li) <= (x[j] + (1 - a[i, j]) * M))
        model.addConstr((x[j] + lj) <= (x[i] + (1 - b[i, j]) * M))
        model.addConstr((y[i] + wi) <= (y[j] + (1 - c[i, j]) * M))
        model.addConstr((y[j] + wj) <= (y[i] + (1 - d[i, j]) * M))
        model.addConstr((z[i] + hi) <= (z[j] + (1 - e[i, j]) * M))
        model.addConstr((z[j] + hj) <= (z[i] + (1 - f[i, j]) * M))

        # Relação espacial por par
        model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= p[i] + p[j] - 1)

# Rodar o modelo
# model.setParam('MIPGap', 0.12) 
model.setParam('TimeLimit', 2)
model.optimize()

packed_volume = model.ObjVal
container_volume = L * W * H
utilization = (packed_volume / container_volume) * 100
print(f"Total volume of packed boxes: {packed_volume}")
print(f"Container volume utilization: {utilization:.2f}%\n")

packed_boxes_indices = [i for i in range(m) if p[i].X > 0.5]
unpacked_boxes_indices = [i for i in range(m) if p[i].X < 0.5]

print(f"Packed {len(packed_boxes_indices)} of {m} boxes.")
print("-" * 30)
aux = 0
for i in packed_boxes_indices:
    # Adjust coordinates to be relative to the container's origin
    rel_x = x[i].X - Xo
    rel_y = y[i].X - Yo
    rel_z = z[i].X - Zo
    aux += vi[i]
    print(f"Box {i+1}: Packed at (x={rel_x:.2f}, y={rel_y:.2f}, z={rel_z:.2f})")

if unpacked_boxes_indices:
    print("\nUnpacked boxes:")
    print(f"Indices: {[i+1 for i in unpacked_boxes_indices]}")

# Criar mapeamento de tipos de caixa
tipo_dict = {}
tipo_counter = 1
for box in boxes:
    if box not in tipo_dict:
        tipo_dict[box] = tipo_counter
        tipo_counter += 1

# Imprimir saída no formato correto
print(f"{L} {W} {H}")  # primeira linha: dimensões do contêiner

for i in range(m):
    li, wi, hi = boxes[i]
    x_i = x[i].X if p[i].X > 0.5 else 0
    y_i = y[i].X if p[i].X > 0.5 else 0
    z_i = z[i].X if p[i].X > 0.5 else 0
    tipo = tipo_dict[boxes[i]]  # tipo real baseado nas dimensões
    cliente = 1
    print(f"{x_i} {y_i} {z_i} {li} {wi} {hi} {tipo} {cliente}")

with open('dados-ahmed.txt','w') as f:
    f.write(f"{L} {W} {H}\n")  # primeira linha: dimensões do contêiner
    for i in range(m):
        if p[i].X > 0.5:
            li, wi, hi = boxes[i]
            x_i, y_i, z_i = x[i].X, y[i].X, z[i].X
            tipo = tipo_dict[boxes[i]]  # tipo baseado nas dimensões
            cliente = 1
            f.write(f"{x_i} {y_i} {z_i} {li} {wi} {hi} {tipo} {cliente}\n")


print(aux*100)

Set parameter TimeLimit to value 2
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  2

Optimize a model with 825 rows, 1410 columns and 2820 nonzeros
Model fingerprint: 0x345c732c
Variable types: 45 continuous, 1365 integer (1365 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [3e-02, 9e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 0.0468750
Presolve removed 138 rows and 768 columns
Presolve time: 0.00s
Presolved: 687 rows, 642 columns, 2538 nonzeros
Variable types: 45 continuous, 597 integer (597 binary)

Root relaxation: objective 1.000000e+00, 223 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   